In [ ]:
import sys
import torch

sys.path.append('/home/local/ASUAD/yzhou298/github/mae')
import models_mae

def prepare_model(chkpt_dir, arch='mae_vit_large_patch16'):
    # build model
    model = getattr(models_mae, arch)()
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cuda')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

# chkpt_dir = 'mae_pretrain_vit_large.pth'
chkpt_dir = 'mae_pretrain_vit_large.pth'
model_mae = prepare_model(chkpt_dir, 'mae_vit_large_patch16')
print(model_mae)

In [ ]:
print(model_mae.forward_encoder)

In [ ]:
from PIL import Image
import numpy as np

img = Image.open("/home/local/ASUAD/yzhou298/Documents/dataset/extended_modattn/put_right_to/split1/0/0.png")
img = np.array(img)[:,:,:3] / 255.

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])
img = img - imagenet_mean
img = img / imagenet_std

# latent, mask, ids_restore = model_mae.forward_encoder(imgs, mask_ratio)img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
img = torch.einsum('nhwc->nchw', img)
print(img.shape)

In [ ]:
latent, mask, ids_restore = model_mae.forward_encoder(img, mask_ratio=0.)
print(latent.shape)
print(mask)
print(ids_restore)

In [ ]:
x_ = latent[:, 1:, :]  # no cls token
x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, latent.shape[2]))  # unshuffle
print(x_.shape)

In [1]:
import sys
import torch

sys.path.append('/home/local/ASUAD/yzhou298/github/mae')
import models_vit

def prepare_model(chkpt_dir, arch='vit_large_patch16'):
    # build model
    model = getattr(models_vit, arch)(global_pool=True)
    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cuda')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

chkpt_dir = 'mae_pretrain_vit_large.pth'
model_mae = prepare_model(chkpt_dir, 'vit_large_patch16')
print(model_mae)

/home/local/ASUAD/yzhou298/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>
VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
 

In [2]:
from PIL import Image
import numpy as np

img = Image.open("/home/local/ASUAD/yzhou298/Documents/dataset/extended_modattn/put_right_to/split1/0/0.png")
img = np.array(img)[:,:,:3] / 255.

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])
img = img - imagenet_mean
img = img / imagenet_std

# latent, mask, ids_restore = model_mae.forward_encoder(imgs, mask_ratio)img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
img = torch.einsum('nhwc->nchw', img)
print(img.shape)

torch.Size([1, 3, 224, 224])


In [3]:
latent = model_mae.forward_features(img)
print(latent.shape)

torch.Size([1, 196, 1024])
